In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import detrend
from statsmodels.tsa.seasonal import seasonal_decompose
import requests

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Cellar/python@3.9/3.9.18/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.9/3.9.18/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/charlesmiller/.local/share/virtualenvs/inv-alerts-pipeline-LK5qlOjP/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_ins

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
KEY = "XpqF6xBLLrj6WALk4SS1UlkgphXmHQec"

class CustomRetry(Retry):
    def is_retry(self, method, status_code, has_retry_after=False):
        """ Return True if we should retry the request, otherwise False. """
        if status_code != 200:
            return True
        return super().is_retry(method, status_code, has_retry_after)
    

def setup_session_retries(
    retries: int = 3,
    backoff_factor: float = 0.05,
    status_forcelist: tuple = (500, 502, 504),
):
    """
    Sets up a requests Session with retries.
    
    Parameters:
    - retries: Number of retries before giving up. Default is 3.
    - backoff_factor: A factor to use for exponential backoff. Default is 0.3.
    - status_forcelist: A tuple of HTTP status codes that should trigger a retry. Default is (500, 502, 504).

    Returns:
    - A requests Session object with retry configuration.
    """
    retry_strategy = CustomRetry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=frozenset(["GET", "POST", "PUT", "DELETE", "HEAD", "OPTIONS"]),
        raise_on_status=False
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

def execute_polygon_call(url):
    session = setup_session_retries()
    response = session.request("GET", url, headers={}, data={})
    return response 

def call_polygon(symbol_list, from_stamp, to_stamp, timespan, multiplier):
    payload={}
    headers = {}
    dfs = []
    
    error_list = []

    if timespan == "minute":
        from_stamp = to_stamp
    for symbol in symbol_list:
        url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={KEY}"

        response = execute_polygon_call(url)

        response_data = json.loads(response.text)
        try:
            results = response_data['results']
        except:
            error_list.append(symbol)
            continue
        results_df = pd.DataFrame(results)
        results_df['t'] = results_df['t'].apply(lambda x: int(x/1000))
        results_df['date'] = results_df['t'].apply(lambda x: convert_timestamp_est(x))
        results_df['hour'] = results_df['date'].apply(lambda x: x.hour)
        results_df['symbol'] = symbol
        dfs.append(results_df)

    return dfs, error_list


In [ ]:
def analyze_volume_trends(data, cycle_length=24):
    # Convert data to numpy array if it's not already
    volume = np.array(data)
    
    # Detrend the data
    detrended = detrend(volume)
    
    # Perform FFT
    fft_result = fft(detrended)
    frequencies = np.fft.fftfreq(len(volume))
    
    # Filter out high-frequency components (adjust threshold as needed)
    threshold = 0.1
    fft_result[np.abs(frequencies) > threshold] = 0
    
    # Inverse FFT to get smoothed trend
    smoothed_trend = np.real(np.fft.ifft(fft_result))
    
    # Perform seasonal decomposition
    df = pd.DataFrame(volume, columns=['volume'])
    decomposition = seasonal_decompose(df, model='additive', period=cycle_length)
    
    # Plot results
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(12, 20))
    
    ax1.plot(volume, label='Original')
    ax1.set_title('Original Volume Data')
    ax1.legend()
    
    ax2.plot(smoothed_trend, label='Smoothed Trend (FFT)')
    ax2.set_title('Smoothed Trend using FFT')
    ax2.legend()
    
    ax3.plot(decomposition.trend, label='Trend (Seasonal Decompose)')
    ax3.set_title('Trend Component from Seasonal Decomposition')
    ax3.legend()
    
    ax4.plot(decomposition.seasonal, label='Seasonal')
    ax4.set_title('Seasonal Component')
    ax4.legend()
    
    ax5.plot(decomposition.resid, label='Residual')
    ax5.set_title('Residual Component (Small Fluctuations)')
    ax5.legend()
    
    plt.tight_layout()
    plt.show()
    
    return smoothed_trend, decomposition